In [3]:
import kotlinx.coroutines.*

runBlocking<Unit> {
    launch {
        println("부모의 콘텍스트 / ${Thread.currentThread().name}")
    }

    launch(Dispatchers.Default) {
        println("Default / ${Thread.currentThread().name}")
    }

    launch(Dispatchers.IO) {
        println("IO / ${Thread.currentThread().name}")
    }

    launch(Dispatchers.Unconfined) {
        println("Unconfined / ${Thread.currentThread().name}")
    }

    launch(newSingleThreadContext("Fast Campus")) {
        println("newSingleThreadContext / ${Thread.currentThread().name}")
    }
}

Unconfined / Execution of code 'import kotlinx.corou...'
부모의 콘텍스트 / Execution of code 'import kotlinx.corou...'


In [4]:
runBlocking<Unit> {
    async {
        println("부모의 콘텍스트 / ${Thread.currentThread().name}")
    }

    async(Dispatchers.Default) {
        println("Default / ${Thread.currentThread().name}")
    }

    async(Dispatchers.IO) {
        println("IO / ${Thread.currentThread().name}")
    }

    async(Dispatchers.Unconfined) {
        println("Unconfined / ${Thread.currentThread().name}")
    }

    async(newSingleThreadContext("Fast Campus")) {
        println("newSingleThreadContext / ${Thread.currentThread().name}")
    }
}

Unconfined / Execution of code 'runBlocking<Unit> {...'
부모의 콘텍스트 / Execution of code 'runBlocking<Unit> {...'


In [5]:
runBlocking<Unit> {
    async(Dispatchers.Unconfined) {
        println("Unconfined / ${Thread.currentThread().name}")
        delay(1000L)
        println("Unconfined / ${Thread.currentThread().name}")
    }
}

Unconfined / Execution of code 'runBlocking<Unit> {...'


In [6]:
runBlocking<Unit> {
    val job = launch {
        launch(Job()) {
            println(coroutineContext[Job])
            println("l1 : ${Thread.currentThread().name}")
            delay(1000L)
            println("3!")
        }


        launch {
            println(coroutineContext[Job])
            println("l2 : ${Thread.currentThread().name}")
            delay(1000L)
            println("1!")
        }
    }

    delay(500L)
    job.cancelAndJoin()
    delay(1000L)
}

StandaloneCoroutine{Active}@564e16d3
l1 : Execution of code 'runBlocking<Unit> {...'
StandaloneCoroutine{Active}@15409ad
l2 : Execution of code 'runBlocking<Unit> {...'
3!


In [8]:
import kotlin.system.measureTimeMillis

runBlocking<Unit> {
    val elapsed = measureTimeMillis {
        val job = launch {
            launch {
                println("l1 : ${Thread.currentThread().name}")
                delay(5_000L)
                println("im done!!")
            }

            launch {
                println("l2 : ${Thread.currentThread().name}")
                delay(10L)
            }
        }

        job.join()
    }

    println("> elapsed : $elapsed")
}

l1 : Execution of code 'import kotlin.system...'
l2 : Execution of code 'import kotlin.system...'
im done!!
> elapsed : 5002


In [11]:
import kotlin.system.*
import kotlinx.coroutines.*
import kotlin.coroutines.*

runBlocking<Unit> {
    launch {
        launch(Dispatchers.IO + CoroutineName("l1")) {
            println("l1: ${Thread.currentThread().name}")
            println(coroutineContext[CoroutineDispatcher])
            println(coroutineContext[CoroutineName])
            delay(5_000L)
        }
    }

    launch(Dispatchers.Default + CoroutineName("l1")) {
        println("l2 : ${Thread.currentThread().name}")
        println(coroutineContext[CoroutineDispatcher])
        println(coroutineContext[CoroutineName])
        delay(10L)
    }
}

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[11], line 9, column 38: This declaration needs opt-in. Its usage must be marked with '@kotlin.ExperimentalStdlibApi' or '@OptIn(kotlin.ExperimentalStdlibApi::class)'
at Cell In[11], line 17, column 34: This declaration needs opt-in. Its usage must be marked with '@kotlin.ExperimentalStdlibApi' or '@OptIn(kotlin.ExperimentalStdlibApi::class)'

In [20]:
import kotlin.random.Random
import kotlin.system.*
import kotlinx.coroutines.*

suspend fun printRandom() {
    delay(500L)
    println(Random.nextInt(0, 500))
}

fun run() {
    val job = GlobalScope.launch(Dispatchers.IO) {
        launch { printRandom() }
    }
    Thread.sleep(1_000L)
}

run()


In [21]:
suspend fun printRandom() {
    delay(500L)
    println(Random.nextInt(0, 500))
}

fun run() {
    val scope = CoroutineScope(Dispatchers.Default)
    val job = scope.launch(Dispatchers.IO) {
        launch { printRandom() }
    }

    Thread.sleep(1_000L)
}

run()

In [26]:
suspend fun printRandom1() {
    delay(1_000L)
    println(Random.nextInt(0, 500))
}

suspend fun printRandom2() {
    delay(500L)
    throw ArithmeticException()
}

val ceh = CoroutineExceptionHandler {_, exception ->
    println("Something happend: ${exception}")
}

runBlocking<Unit> {
    val scope = CoroutineScope(Dispatchers.IO)
    val job = scope.launch (ceh) {
        launch { printRandom1() }
        launch { printRandom2() }
    }

    job.join()
}

In [31]:
suspend fun getRandom1(): Int {
    delay(1_000L)
    return Random.nextInt(0, 500)
}

suspend fun getRandom2(): Int {
    delay(500L)
    throw ArithmeticException()
}

val ceh = CoroutineExceptionHandler { _, exception ->
    println("Something happend!! : ${exception}")
}

runBlocking <Unit> {
    val job = launch (ceh) {
        val a = async { getRandom1() }
        val b = async { getRandom2() }
        println(a.await())
        println(b.await())
    }
    job.join()
}

java.lang.ArithmeticException: 

In [32]:
suspend fun printRandom1() {
    delay(1000L)
    println(Random.nextInt(0, 500))
}

suspend fun printRandom2() {
    delay(500L)
    throw ArithmeticException()
}

val ceh = CoroutineExceptionHandler { _, exception ->
    println("Something Happend!! : $exception")
}

runBlocking<Unit> {
    val scope = CoroutineScope(Dispatchers.IO + SupervisorJob() + ceh)
    val job1 = scope.launch { printRandom1() }
    val job2 = scope.launch { printRandom2() }

    joinAll(job1, job2)
}

In [33]:
suspend fun printRandom1() {
    delay(1_000L)
    println(Random.nextInt(0, 500))
}

suspend fun printRandom2() {
    delay(500L)
    throw ArithmeticException()
}

suspend fun supervisoredFunc() = supervisorScope {
    launch { printRandom1() }
    launch { printRandom2() }
}

val ceh = CoroutineExceptionHandler { _, exception ->
    println("Something happed: $exception")
}

runBlocking<Unit> {
    val scope = CoroutineScope(Dispatchers.IO)
    val job = scope.launch {
        supervisoredFunc()
    }
    job.join()
}

In [40]:
import kotlin.system.*
import kotlinx.coroutines.*


suspend fun massiveRun(action: suspend () -> Unit) {
    val n = 100 // 시작할 코루틴의 갯수
    val k = 1000 // 코루틴 내에서 반복할 횟수
    val elapsed = measureTimeMillis {
        coroutineScope { // scope for coroutines
            repeat(n) {
                launch {
                    repeat(k) { action() }
                }
            }
        }
    }
    println("$elapsed ms동안 ${n * k}개의 액션을 수행했습니다.")
}

var counter = 0

runBlocking {
    withContext(Dispatchers.Default) {
        massiveRun {
            counter++
        }
    }

    println("Couter = ${counter}")
}

Couter = 58301


In [44]:
suspend fun massiveRun(action: suspend () -> Unit) {
    val n = 100 // 시작할 코루틴의 갯수
    val k = 1000 // 코루틴 내에서 반복할 횟수
    val elapsed = measureTimeMillis {
        coroutineScope { // scope for coroutines
            repeat(n) {
                launch {
                    repeat(k) { action() }
                }
            }
        }
    }
    println("$elapsed ms동안 ${n * k}개의 액션을 수행했습니다.")
}

@Volatile
var counter = 0 // volatile은 가시성 문제만을 해결할 뿐 동시에 읽고 수정해서 생기는 문제를 해결하지 못함

runBlocking {
    withContext(Dispatchers.Default) {
        massiveRun {
            counter++
        }
    }

    println("Couter = $counter")
}

Couter = 52353


In [45]:
import java.util.concurrent.atomic.*
import kotlin.system.*
import kotlinx.coroutines.*


suspend fun massiveRun(action: suspend () -> Unit) {
    val n = 100 // 시작할 코루틴의 갯수
    val k = 1000 // 코루틴 내에서 반복할 횟수
    val elapsed = measureTimeMillis {
        coroutineScope { // scope for coroutines
            repeat(n) {
                launch {
                    repeat(k) { action() }
                }
            }
        }
    }
    println("$elapsed ms동안 ${n * k}개의 액션을 수행했습니다.")
}

val counter = AtomicInteger()

runBlocking {
    withContext(Dispatchers.Default) {
        massiveRun {
            counter.incrementAndGet()
        }
    }
    println("Counter = $counter")
}

Counter = 100000


In [47]:
suspend fun massiveRun(action: suspend () -> Unit) {
    val n = 100 // 시작할 코루틴의 갯수
    val k = 1000 // 코루틴 내에서 반복할 횟수
    val elapsed = measureTimeMillis {
        coroutineScope { // scope for coroutines
            repeat(n) {
                launch {
                    repeat(k) { action() }
                }
            }
        }
    }
    println("$elapsed ms동안 ${n * k}개의 액션을 수행했습니다.")
}

var counter = 0
val counterContext = newSingleThreadContext("CounterContext") //  특정한 스레드를 만들고 해당 스레드를 사용

runBlocking {
    withContext(counterContext) {
        massiveRun {
            counter++
        }
    }
    println("Counter = $counter")
}

Counter = 100000


In [48]:
import kotlin.system.*
import kotlinx.coroutines.*
import kotlinx.coroutines.sync.*

suspend fun massiveRun(action: suspend () -> Unit) {
    val n = 100 // 시작할 코루틴의 갯수
    val k = 1000 // 코루틴 내에서 반복할 횟수
    val elapsed = measureTimeMillis {
        coroutineScope { // scope for coroutines
            repeat(n) {
                launch {
                    repeat(k) { action() }
                }
            }
        }
    }
    println("$elapsed ms동안 ${n * k}개의 액션을 수행했습니다.")
}

val mutex = Mutex() // 유 상태를 수정할 때 임계 영역(critical section)를 이용하게 하며, 임계 영역을 동시에 접근하는 것을 허용하지 않음
var counter = 0

runBlocking {
    withContext(Dispatchers.Default) {
        massiveRun {
            mutex.withLock {
                counter++
            }
        }
    }
    println("Counter = $counter")
}

Counter = 100000


In [51]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*
import kotlin.system.*


suspend fun massiveRun(action: suspend () -> Unit) {
    val n = 100
    val k = 1000
    val elapsed = measureTimeMillis {
        coroutineScope {
            repeat(n) {
                launch {
                    repeat(k) { action() }
                }
            }
        }
    }
    println("$elapsed ms동안 ${n * k}개의 액션을 수행했습니다.")
}


sealed class CounterMsg
object IncCounter : CounterMsg()
class GetCounter(val response: CompletableDeferred<Int>) : CounterMsg()

fun CoroutineScope.counterActor() = actor<CounterMsg> {
    var counter = 0
    for (msg in channel) {
        when (msg) {
            is IncCounter -> counter++
            else -> msg.response.complete(counter) // is GetCounter
        }
    }
}


runBlocking<Unit> {
    val counter = counterActor()
    withContext(Dispatchers.Default) {
        massiveRun {
            counter.send(IncCounter)
        }
    }

    val response = CompletableDeferred<Int>()
    counter.send(GetCounter(response))
    println("Counter = ${response.await()}")
    counter.close()
}

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[51], line 26, column 37: This declaration needs opt-in. Its usage should be marked with '@kotlinx.coroutines.ObsoleteCoroutinesApi' or '@OptIn(kotlinx.coroutines.ObsoleteCoroutinesApi::class)'
at Cell In[51], line 28, column 17: This declaration needs opt-in. Its usage should be marked with '@kotlinx.coroutines.ObsoleteCoroutinesApi' or '@OptIn(kotlinx.coroutines.ObsoleteCoroutinesApi::class)'
at Cell In[51], line 31, column 25: Unresolved reference: response